In [133]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [134]:
train_trn = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv', index_col='TransactionID')
test_trn = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv', index_col='TransactionID')

train_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv', index_col='TransactionID')
test_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv', index_col='TransactionID')

In [135]:
print(train_id.shape, test_id.shape)
print(train_trn.shape, test_trn.shape)

(144233, 40) (141907, 40)
(590540, 393) (506691, 392)


In [136]:
train_trn.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
train_id.head()

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [138]:
test_id.head()

,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,...,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [139]:
print("percent of fraudulent transactions in train dataset: ", len(train_trn.loc[train_trn.isFraud == 1])*100/len(train_trn))

percent of fraudulent transactions in train dataset:  3.499000914417313


In [140]:
y_train = train_trn["isFraud"]

# drop target column from train dataframe
train_trn = train_trn.drop(columns = ['isFraud'])

print(y_train.shape)
print(train_trn.shape)

(590540,)
(590540, 392)


In [141]:

test_id.columns = ['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08',
       'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16',
       'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24',
       'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo']


print("before concatting: \n\n")
print("train_transaction.shape: ", train_trn.shape, "\n")
print("test_transaction.shape: ", test_trn.shape, "\n")
print("train_transaction.index: ", train_trn.index, "\n")
print("test_transaction.index: ", test_trn.index, "\n")

print("train_identity.shape: ", train_id.shape, "\n")
print("test_identity.shape: ", test_id.shape, "\n")
print("train_identity.index: ", train_id.index, "\n")
print("test_identity.index: ", test_id.index, "\n")



trn_data = pd.concat([train_trn, test_trn])
id_data = pd.concat([train_id, test_id])
print("after concatting: \n\n")
print("transaction_data.shape: ", trn_data.shape, "\n")
print("transaction_data.index: ", trn_data.index, "\n")
print("identity_data.shape: ", id_data.shape, "\n")
print("identity_data.index: ", id_data.index, "\n")

before concatting: 


train_transaction.shape:  (590540, 392) 

test_transaction.shape:  (506691, 392) 

train_transaction.index:  Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            3577530, 3577531, 3577532, 3577533, 3577534, 3577535, 3577536,
            3577537, 3577538, 3577539],
           dtype='int64', name='TransactionID', length=590540) 

test_transaction.index:  Int64Index([3663549, 3663550, 3663551, 3663552, 3663553, 3663554, 3663555,
            3663556, 3663557, 3663558,
            ...
            4170230, 4170231, 4170232, 4170233, 4170234, 4170235, 4170236,
            4170237, 4170238, 4170239],
           dtype='int64', name='TransactionID', length=506691) 

train_identity.shape:  (144233, 40) 

test_identity.shape:  (141907, 40) 

train_identity.index:  Int64Index([2987004, 2987008, 2987010, 2987011, 2987016, 2987017, 2987022,
            2987038, 2987040, 2987048,
            

In [142]:
cat = (id_data.dtypes == 'object')
num = (id_data.dtypes != 'object')
cat_id_cols = list(cat[cat].index)
num_id_cols = list(num[num].index) 

print(cat_id_cols, "\n")
print("number categorical identity features: ", len(cat_id_cols), "\n\n")
print(num_id_cols, "\n")
print("number numerical identity features: ", len(num_id_cols))

['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo'] 

number categorical identity features:  17 


['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32'] 

number numerical identity features:  23


In [143]:
cat = (trn_data.dtypes == 'object')
num = (trn_data.dtypes != 'object')
cat_trans_cols = list(cat[cat].index)
num_trans_cols = list(num[num].index) 

print(cat_trans_cols, "\n")
print("number categorical transaction features: ", len(cat_trans_cols), "\n\n")
print(num_trans_cols, "\n")
print("number numerical transaction features: ", len(num_trans_cols))

['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'] 

number categorical transaction features:  14 


['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 

In [144]:
shape_of_train_trans = train_trn.shape
shape_of_train_id    = train_id.shape

shape_of_test_trans  = test_trn.shape
shape_of_test_id     = test_id.shape

del train_trn
del train_id
del test_trn
del test_id

In [145]:
for i in cat_id_cols:
    print(id_data[i].value_counts())
    print(i, "missing values: ", id_data[i].isnull().sum())
    print(id_data[i].isnull().sum()*100/ len(id_data[i]), "\n")

NotFound    243920
Found        42220
Name: id_12, dtype: int64
id_12 missing values:  0
0.0 

Found      135690
New        119397
Unknown     22875
Name: id_15, dtype: int64
id_15 missing values:  8178
2.8580415181379744 

Found       132805
NotFound    122282
Name: id_16, dtype: int64
id_16 missing values:  31053
10.852379953868736 

IP_PROXY:TRANSPARENT    7203
IP_PROXY:ANONYMOUS      2010
IP_PROXY:HIDDEN         1018
Name: id_23, dtype: int64
id_23 missing values:  275909
96.42447752848256 

Found       10214
NotFound       17
Name: id_27, dtype: int64
id_27 missing values:  275909
96.42447752848256 

Found    151813
New      125943
Name: id_28, dtype: int64
id_28 missing values:  8384
2.930034248969036 

Found       149264
NotFound    128492
Name: id_29, dtype: int64
id_29 missing values:  8384
2.930034248969036 

Windows 10          42170
Windows 7           23478
iOS 12.1.0           6349
Mac OS X 10_12_6     3884
iOS 11.2.1           3824
                    ...  
Mac OS X 10_1

In [146]:
low_missing_cat_id_cols = []      # lower than 15% missing values
medium_missing_cat_id_cols = []   # between 15% and 60% missing
many_missing_cat_id_cols = []     # more than 60% missing

for i in cat_id_cols:
    percentage = id_data[i].isnull().sum() * 100 / len(id_data[i])
    if percentage < 15:
        low_missing_cat_id_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_cat_id_cols.append(i)
    else:
        many_missing_cat_id_cols.append(i)
        
print("cat_id_cols: \n\n")      
print("number low missing: ", len(low_missing_cat_id_cols), "\n")
print("number medium missing: ", len(medium_missing_cat_id_cols), "\n")
print("number many missing: ", len(many_missing_cat_id_cols), "\n")

cat_id_cols: 


number low missing:  11 

number medium missing:  4 

number many missing:  2 



In [147]:
for i in num_id_cols:
    print(id_data[i].value_counts())
    print(i, "missing values: ", id_data[i].isnull().sum()) 
    print(id_data[i].isnull().sum()*100/len(id_data[i]), "\n")

-5.0     164749
 0.0      31555
-10.0     23050
-20.0     22341
-15.0     12722
          ...  
-82.0         1
-89.0         1
-24.0         1
-94.0         1
-77.0         1
Name: id_01, Length: 89, dtype: int64
id_01 missing values:  0
0.0 

696.0       16
1083.0      15
552.0       15
1141.0      12
1102.0      12
            ..
267094.0     1
177766.0     1
382330.0     1
279629.0     1
692090.0     1
Name: id_02, Length: 198051, dtype: int64
id_02 missing values:  8292
2.8978821555881735 

 0.0     127811
 1.0       1645
 3.0       1442
 2.0        806
 5.0        264
 4.0        184
 6.0        134
-5.0         88
-6.0         84
-2.0         58
-4.0         56
-7.0         54
-8.0         41
-9.0         27
-10.0        26
-11.0        20
-12.0        19
-1.0         18
-3.0         12
 7.0          6
 9.0          4
-13.0         3
 10.0         1
 8.0          1
 11.0         1
Name: id_03, dtype: int64
id_03 missing values:  153335
53.58740476689732 

 0.0     131390
-5.0   

In [148]:
low_missing_num_id_cols = []      # lower than 15% missing values
medium_missing_num_id_cols = []   # between 15% and 60% missing
many_missing_num_id_cols = []     # more than 60% missing

for i in num_id_cols:
    percentage = id_data[i].isnull().sum() * 100 / len(id_data[i])
    if percentage < 15:
        low_missing_num_id_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_num_id_cols.append(i)
    else:
        many_missing_num_id_cols.append(i)
        
print("num_id_cols: \n\n")        
print("number low missing: ", len(low_missing_num_id_cols), "\n")
print("number medium missing: ", len(medium_missing_num_id_cols), "\n")
print("number many missing: ", len(many_missing_num_id_cols), "\n")

num_id_cols: 


number low missing:  9 

number medium missing:  6 

number many missing:  8 



In [149]:
for i in cat_trans_cols:
    print(trn_data[i].value_counts())
    print(i, trn_data[i].isnull().sum(), "missing values")
    print(i, trn_data[i].isnull().sum()*100/len(trn_data[i]), "\n") 

W    800657
C    137785
R     73346
H     62397
S     23046
Name: ProductCD, dtype: int64
ProductCD 0 missing values
ProductCD 0.0 

visa                719649
mastercard          347386
american express     16009
discover              9524
Name: card4, dtype: int64
card4 4663 missing values
card4 0.42497887864998346 

debit              824959
credit             267648
debit or credit        30
charge card            16
Name: card6, dtype: int64
card6 4578 missing values
card6 0.41723210518113324 

gmail.com           435803
yahoo.com           182784
hotmail.com          85649
anonymous.com        71062
aol.com              52337
comcast.net          14474
icloud.com           12316
outlook.com           9934
att.net               7647
msn.com               7480
sbcglobal.net         5767
live.com              5720
verizon.net           5011
ymail.com             4075
bellsouth.net         3437
yahoo.com.mx          2827
me.com                2713
cox.net               2657
optonline

In [150]:
low_missing_num_trans_cols = []      # lower than 15% missing values
medium_missing_num_trans_cols = []   # between 15% and 60% missing
many_missing_num_trans_cols = []     # more than 60% missing

for i in num_trans_cols:
    percentage = trn_data[i].isnull().sum() * 100 / len(trn_data[i])
    if percentage < 15:
        low_missing_num_trans_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_num_trans_cols.append(i)
    else:
        many_missing_num_trans_cols.append(i)
        
print("num_trans_cols: \n\n")        
print("number low missing: ", len(low_missing_num_trans_cols), "\n")
print("number medium missing: ", len(medium_missing_num_trans_cols), "\n")
print("number many missing: ", len(many_missing_num_trans_cols), "\n")

num_trans_cols: 


number low missing:  176 

number medium missing:  35 

number many missing:  167 



In [151]:
low_missing_cat_trans_cols = []      # lower than 15% missing values
medium_missing_cat_trans_cols = []   # between 15% and 60% missing
many_missing_cat_trans_cols = []     # more than 60% missing

for i in cat_trans_cols:
    percentage = trn_data[i].isnull().sum() * 100 / len(trn_data[i])
    if percentage < 15:
        low_missing_cat_trans_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_cat_trans_cols.append(i)
    else:
        many_missing_cat_trans_cols.append(i)
        
print("cat_trans_cols: \n\n")    
print("number low missing: ", len(low_missing_cat_trans_cols), "\n")
print("number medium missing: ", len(medium_missing_cat_trans_cols), "\n")
print("number many missing: ", len(many_missing_cat_trans_cols), "\n")

cat_trans_cols: 


number low missing:  4 

number medium missing:  8 

number many missing:  2 



In [152]:
print("shape before dropping num_trans_cols: ", trn_data.shape, "\n")        
trn_data = trn_data.drop(columns = many_missing_num_trans_cols)
print("shape after dropping num_trans_cols: ", trn_data.shape, "\n\n")    


print("shape before dropping num_id_cols: ", id_data.shape, "\n")        
id_data = id_data.drop(columns = many_missing_num_id_cols)
print("shape after dropping num_id_cols: ", id_data.shape, "\n")


# because we dropped some numerical columns from the dataframe,
# we must create the list 'num_trans_cols' and
# 'num_id_cols' again such that the dropped cols are no longer in them
num = (trn_data.dtypes != 'object')
num_trans_cols = list(num[num].index) 

num = (id_data.dtypes != 'object')
num_id_cols = list(num[num].index)

shape before dropping num_trans_cols:  (1097231, 392) 

shape after dropping num_trans_cols:  (1097231, 225) 


shape before dropping num_id_cols:  (286140, 40) 

shape after dropping num_id_cols:  (286140, 32) 



In [153]:
#Imputation
from sklearn.impute import SimpleImputer

#print("index before imputation: ", trn_data.index, "\n")
#print("columns before imputation: ", trn_data.columns, "\n")

#print("starting imputation..... \n\n")
my_imputer = SimpleImputer(strategy = 'mean') 
my_imputer.fit(trn_data[low_missing_num_trans_cols])

#print("values before imputing: ", train_transaction[low_missing_num_trans_cols], "\n")

trn_data[low_missing_num_trans_cols] = my_imputer.transform(trn_data[low_missing_num_trans_cols])

#print("index after imputation: ", trn_data.index, "\n")
#print("columns after imputation: ", trn_data.columns, "\n")

print("values after imputing: ", trn_data[low_missing_num_trans_cols], "\n")

print("As we can see the imputation was successful! \n")

values after imputing:                 TransactionDT  TransactionAmt    card1       card2  card3  \
TransactionID                                                              
2987000              86400.0          68.500  13926.0  363.099769  150.0   
2987001              86401.0          29.000   2755.0  404.000000  150.0   
2987002              86469.0          59.000   4663.0  490.000000  150.0   
2987003              86499.0          50.000  18132.0  567.000000  150.0   
2987004              86506.0          50.000   4497.0  514.000000  150.0   
...                      ...             ...      ...         ...    ...   
4170235           34214279.0          94.679  13832.0  375.000000  185.0   
4170236           34214287.0          12.173   3154.0  408.000000  185.0   
4170237           34214326.0          49.000  16661.0  490.000000  150.0   
4170238           34214337.0         202.000  16621.0  516.000000  150.0   
4170239           34214345.0          24.346   5713.0  168.00000

In [154]:
#print("index before imputation: ", id_data.index, "\n")
#print("columns before imputation: ", id_data.columns, "\n")


my_imputer = SimpleImputer(strategy = 'mean') 
my_imputer.fit(id_data[low_missing_num_id_cols])

#print("starting imputation....\n")
id_data[low_missing_num_id_cols] = my_imputer.transform(id_data[low_missing_num_id_cols])

#print("index after imputation: ", id_data.index, "\n")
#print("columns after imputation: ", id_data.columns, "\n")

In [155]:
#print("index before imputation: ", trn_data.index, "\n")
#print("columns before imputation: ", trn_data.columns, "\n")

#print("values before imputing: ", trn_data[medium_missing_num_trans_cols], "\n")

#print("starting imputation.....\n\n")
my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(trn_data[medium_missing_num_trans_cols])

trn_data[medium_missing_num_trans_cols] = my_imputer.transform(trn_data[medium_missing_num_trans_cols])

#print("index after imputation: ", trn_data.index, "\n")
#print("columns after imputation: ", trn_data.columns, "\n")

print("values after imputing: ", trn_data[medium_missing_num_trans_cols], "\n")

values after imputing:                 dist1     D2    D3    D4    D5    D11   V1   V2   V3   V4  ...  \
TransactionID                                                             ...   
2987000         19.0  104.0  13.0  23.0   9.0   13.0  1.0  1.0  1.0  1.0  ...   
2987001          8.0  104.0   8.0   0.0   9.0   69.0  1.0  1.0  1.0  1.0  ...   
2987002        287.0  104.0   8.0   0.0   9.0  315.0  1.0  1.0  1.0  1.0  ...   
2987003          8.0  112.0   0.0  94.0   0.0   69.0  1.0  1.0  1.0  1.0  ...   
2987004          8.0  104.0   8.0  23.0   9.0   69.0  1.0  1.0  1.0  1.0  ...   
...              ...    ...   ...   ...   ...    ...  ...  ...  ...  ...  ...   
4170235          8.0  104.0   8.0   0.0   9.0   69.0  1.0  1.0  1.0  1.0  ...   
4170236          8.0   16.0  16.0  16.0  16.0   69.0  1.0  1.0  1.0  1.0  ...   
4170237          8.0  104.0   8.0   0.0   9.0    0.0  1.0  1.0  1.0  1.0  ...   
4170238          8.0  104.0   8.0   0.0   9.0    0.0  1.0  1.0  1.0  1.0  ...   
4170

In [156]:
print("index before imputation: ", id_data.index, "\n")
print("columns before imputation: ", id_data.columns, "\n")


my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(id_data[medium_missing_num_id_cols])

print("values before imputing: ", id_data[medium_missing_num_id_cols], "\n")

id_data[medium_missing_num_id_cols] = my_imputer.transform(id_data[medium_missing_num_id_cols])

print("index after imputation: ", id_data.index, "\n")
print("columns after imputation: ", id_data.columns, "\n")

index before imputation:  Int64Index([2987004, 2987008, 2987010, 2987011, 2987016, 2987017, 2987022,
            2987038, 2987040, 2987048,
            ...
            4170224, 4170225, 4170226, 4170228, 4170229, 4170230, 4170233,
            4170234, 4170236, 4170239],
           dtype='int64', name='TransactionID', length=286140) 

columns before imputation:  Index(['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
       'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_19',
       'id_20', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object') 

values before imputing:                 id_03  id_04  id_09  id_10  id_14  id_32
TransactionID                                          
2987004          NaN    NaN    NaN    NaN -480.0   32.0
2987008          NaN    NaN    NaN    NaN -300.0   32.0
2987010          0.0    0.0  

In [157]:
print(trn_data[num_trans_cols].isnull().sum().sum())
print(id_data[num_id_cols].isnull().sum().sum())

0
0


In [158]:
# Memory Usage
print("transaction_data.memory_usage(): ", trn_data.info(), "\n")  

print("identity_data.memory_usage(): ", id_data.info(), "\n") 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Columns: 225 entries, TransactionDT to V321
dtypes: float64(211), object(14)
memory usage: 1.8+ GB
transaction_data.memory_usage():  None 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 32 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_01       286140 non-null  float64
 1   id_02       286140 non-null  float64
 2   id_03       286140 non-null  float64
 3   id_04       286140 non-null  float64
 4   id_05       286140 non-null  float64
 5   id_06       286140 non-null  float64
 6   id_09       286140 non-null  float64
 7   id_10       286140 non-null  float64
 8   id_11       286140 non-null  float64
 9   id_12       286140 non-null  object 
 10  id_13       286140 non-null  float64
 11  id_14       286140 non-null  float64
 12  id_15       277962 non-null  object 
 13  id_16       255087 no

In [159]:
object_counter = 0
int_counter = 0
float_counter = 0

not_detected = []

for i in trn_data.columns:
        if trn_data[i].dtype == 'object':
            object_counter += 1
        elif trn_data[i].dtype == 'int':
            int_counter += 1
        elif trn_data[i].dtype in ['float', 'float16', 'float32', 'float64']:
            float_counter += 1
        else:
            not_detected.append(i)
            
print("transaction_data has ", "\n")
print(object_counter, "object columns, \n")
print(int_counter, "int columns, \n")
print(float_counter, "float columns \n")

total = object_counter + int_counter  + float_counter

if total != len(trn_data.columns):
    
    print("D DOUBLE DANGER: some columns have not been detected!!")
    print("these columns have not been detected: ", not_detected)
    for i in not_detected:
        print(trn_data[i].dtype, "\n")

transaction_data has  

14 object columns, 

0 int columns, 

211 float columns 



In [160]:
object_counter = 0
int_counter = 0
float_counter = 0

not_detected = []

for i in id_data.columns:
        if id_data[i].dtype == 'object':
            object_counter += 1
        elif id_data[i].dtype == 'int':
            int_counter += 1
        elif id_data[i].dtype in ['float', 'float16', 'float32', 'float64']:
            float_counter += 1
        else:
            not_detected.append(i)
            
            
print("identity_data has ", "\n")
print(object_counter, "object columns, \n")
print(int_counter, "int columns, \n")
print(float_counter, "float columns \n")

total = object_counter + int_counter  + float_counter

if total != len(id_data.columns):
    
    print("D DOUBLE DANGER: some columns have not been detected!!")
    print("these columns have not been detected: ", not_detected)    
    for i in not_detected:
        print(id_data[i].dtype, "\n")

identity_data has  

17 object columns, 

0 int columns, 

15 float columns 



In [161]:
#  this function detects all the numerical columns,
#  that can be converted to a smaller datatype.

def detect_num_cols_to_shrink(list_of_num_cols, dataframe):
 
    convert_to_int8 = []
    convert_to_int16 = []
    convert_to_int32 = []
    
    #  sadly the datatype float8 does not exist
    convert_to_float16 = []
    convert_to_float32 = []
    
    for col in list_of_num_cols:
        
        if dataframe[col].dtype in ['int', 'int8', 'int32', 'int64']:
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 255:
                convert_to_int8.append(col)
            elif diff < 65535:
                convert_to_int16.append(col)
            elif diff < 4294967295:
                convert_to_int32.append(col)   
                
        elif dataframe[col].dtype in ['float', 'float16', 'float32', 'float64']:
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 65535:
                convert_to_float16.append(col)
            elif diff < 4294967295:
                convert_to_float32.append(col) 
        
    list_of_lists = []
    list_of_lists.append(convert_to_int8)
    list_of_lists.append(convert_to_int16)
    list_of_lists.append(convert_to_int32)
    list_of_lists.append(convert_to_float16)
    list_of_lists.append(convert_to_float32)
    
    return list_of_lists

In [162]:
num_cols_to_shrink_trans = detect_num_cols_to_shrink(num_trans_cols, trn_data)

convert_to_int8 = num_cols_to_shrink_trans[0]
convert_to_int16 = num_cols_to_shrink_trans[1]
convert_to_int32 = num_cols_to_shrink_trans[2]

convert_to_float16 = num_cols_to_shrink_trans[3]
convert_to_float32 = num_cols_to_shrink_trans[4]

print("convert_to_int8 :", convert_to_int8, "\n")
print("convert_to_int16 :", convert_to_int16, "\n")
print("convert_to_int32 :", convert_to_int32, "\n")

print("convert_to_float16 :", convert_to_float16, "\n")
print("convert_to_float32 :", convert_to_float32, "\n")

convert_to_int8 : [] 

convert_to_int16 : [] 

convert_to_int32 : [] 

convert_to_float16 : ['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V

In [163]:
print("starting with converting process....")

for col in convert_to_int16:
    trn_data[col] = trn_data[col].astype('int16') 
    
for col in convert_to_int32:
    trn_data[col] = trn_data[col].astype('int32') 

for col in convert_to_float16:
    trn_data[col] = trn_data[col].astype('float16')
    
for col in convert_to_float32:
    trn_data[col] = trn_data[col].astype('float32')
    
print("successfully converted!")

starting with converting process....
successfully converted!


In [164]:
num_cols_to_shrink_id = detect_num_cols_to_shrink(num_id_cols, id_data)

convert_to_int8 = num_cols_to_shrink_id[0]
convert_to_int16 = num_cols_to_shrink_id[1]
convert_to_int32 = num_cols_to_shrink_id[2]

convert_to_float16 = num_cols_to_shrink_id[3]
convert_to_float32 = num_cols_to_shrink_id[4]

print("convert_to_int8 :", convert_to_int8, "\n")
print("convert_to_int16 :", convert_to_int16, "\n")
print("convert_to_int32 :", convert_to_int32, "\n")

print("convert_to_float16 :", convert_to_float16, "\n")
print("convert_to_float32 :", convert_to_float32, "\n")

convert_to_int8 : [] 

convert_to_int16 : [] 

convert_to_int32 : [] 

convert_to_float16 : ['id_01', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_19', 'id_20', 'id_32'] 

convert_to_float32 : ['id_02'] 



In [165]:
for col in convert_to_float16:
    id_data[col] = id_data[col].astype('float16')
    
for col in convert_to_float32:
    id_data[col] = id_data[col].astype('float32')
    
    
print("successfully converted!")

successfully converted!


In [166]:
print("transaction_data.memory_usage(): ", trn_data.info(), "\n")   

print("identity_data.memory_usage(): ", id_data.info(), "\n") 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Columns: 225 entries, TransactionDT to V321
dtypes: float16(188), float32(23), object(14)
memory usage: 615.3+ MB
transaction_data.memory_usage():  None 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 32 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_01       286140 non-null  float16
 1   id_02       286140 non-null  float32
 2   id_03       286140 non-null  float16
 3   id_04       286140 non-null  float16
 4   id_05       286140 non-null  float16
 5   id_06       286140 non-null  float16
 6   id_09       286140 non-null  float16
 7   id_10       286140 non-null  float16
 8   id_11       286140 non-null  float16
 9   id_12       286140 non-null  object 
 10  id_13       286140 non-null  float16
 11  id_14       286140 non-null  float16
 12  id_15       277962 non-null  object 
 13  id_16 

In [167]:
# Drop cat. features with many missing values
print("shape before dropping many_missing_cat_trans_cols: ", trn_data.shape, "\n")        
trn_data = trn_data.drop(columns = many_missing_cat_trans_cols)
print("shape after dropping many_missing_cat_trans_cols: ", trn_data.shape, "\n\n")    

print("shape before dropping many_missing_cat_id_cols: ", id_data.shape, "\n")        
id_data = id_data.drop(columns = many_missing_cat_id_cols)
print("shape after dropping many_missing_cat_id_cols: ", id_data.shape, "\n")


# because we dropped some categorical columns from the dataframe,
# we must create the list 'cat_trans_cols' and
# 'cat_id_cols' again such that the dropped cols are no longer in them

cat = (trn_data.dtypes == 'object')
cat_trans_cols = list(cat[cat].index) 

cat = (id_data.dtypes == 'object')
cat_id_cols = list(cat[cat].index) 

shape before dropping many_missing_cat_trans_cols:  (1097231, 225) 

shape after dropping many_missing_cat_trans_cols:  (1097231, 223) 


shape before dropping many_missing_cat_id_cols:  (286140, 32) 

shape after dropping many_missing_cat_id_cols:  (286140, 30) 



In [168]:
# Label Encoder
for col in cat_trans_cols:
    print(col, trn_data[col].nunique(), "\n")

ProductCD 5 

card4 4 

card6 4 

P_emaildomain 60 

M1 2 

M2 2 

M3 2 

M4 3 

M6 2 

M7 2 

M8 2 

M9 2 



In [169]:
low_card_trans_cols = ["ProductCD", "card4", "card6", "M1", "M2", "M3", "M4", "M6", "M7", "M8", "M9"]
high_card_trans_cols = ["P_emaildomain"]

print("lists successfully created!")

lists successfully created!


In [170]:
#  label-encode our high cardinality features.
# replace all NaN's with the most frequent value per columns.
for i in cat_trans_cols:
    most_frequent_value = trn_data[i].mode()[0]
    print("For column: ", i, "the most frequent value is: ", most_frequent_value, "\n")
    trn_data[i].fillna(most_frequent_value, inplace = True)

For column:  ProductCD the most frequent value is:  W 

For column:  card4 the most frequent value is:  visa 

For column:  card6 the most frequent value is:  debit 

For column:  P_emaildomain the most frequent value is:  gmail.com 

For column:  M1 the most frequent value is:  T 

For column:  M2 the most frequent value is:  T 

For column:  M3 the most frequent value is:  T 

For column:  M4 the most frequent value is:  M0 

For column:  M6 the most frequent value is:  F 

For column:  M7 the most frequent value is:  F 

For column:  M8 the most frequent value is:  F 

For column:  M9 the most frequent value is:  T 



In [171]:
from sklearn.preprocessing import LabelEncoder
    
label_encoder = LabelEncoder()
print("transaction_data.shape before label-encoding: ", trn_data.shape, "\n")

#trn_data[high_card_trans_cols] = label_encoder.fit_transform(trn_data[high_card_trans_cols])

print("transaction_data.shape after label-encoding: ", trn_data.shape, "\n")
#print("transaction_data[high_card_trans_cols] after label_encoding: ",trn_data[high_card_trans_cols],"\n")

transaction_data.shape before label-encoding:  (1097231, 223) 

transaction_data.shape after label-encoding:  (1097231, 223) 



In [172]:
for col in cat_id_cols:
    print(col, id_data[col].nunique(), "\n")

id_12 2 

id_15 3 

id_16 2 

id_28 2 

id_29 2 

id_30 87 

id_31 172 

id_33 461 

id_34 4 

id_35 2 

id_36 2 

id_37 2 

id_38 2 

DeviceType 2 

DeviceInfo 2799 



In [173]:
low_card_id_cols =  ["id_12", "id_15", "id_16", "id_28", "id_29", "id_34", "id_35", "id_36", "id_37", "id_38", "DeviceType"]
high_card_id_cols = ["id_30", "id_31", "id_33", "DeviceInfo"]
    
print("lists successfully created!")

lists successfully created!


In [174]:
for i in cat_id_cols:
    most_frequent_value = id_data[i].mode()[0]
    print("For column: ", i, "the most frequent value is: ", most_frequent_value, "\n")
    id_data[i].fillna(most_frequent_value, inplace = True)

For column:  id_12 the most frequent value is:  NotFound 

For column:  id_15 the most frequent value is:  Found 

For column:  id_16 the most frequent value is:  Found 

For column:  id_28 the most frequent value is:  Found 

For column:  id_29 the most frequent value is:  Found 

For column:  id_30 the most frequent value is:  Windows 10 

For column:  id_31 the most frequent value is:  mobile safari 11.0 

For column:  id_33 the most frequent value is:  1920x1080 

For column:  id_34 the most frequent value is:  match_status:2 

For column:  id_35 the most frequent value is:  T 

For column:  id_36 the most frequent value is:  F 

For column:  id_37 the most frequent value is:  T 

For column:  id_38 the most frequent value is:  F 

For column:  DeviceType the most frequent value is:  desktop 

For column:  DeviceInfo the most frequent value is:  Windows 



In [175]:
label_encoder = LabelEncoder()

print("identity_data.shape before label-encoding: ", id_data.shape, "\n")

for col in high_card_id_cols:
    id_data[col] = label_encoder.fit_transform(id_data[col])

print("identity_data.shape after label-encoding: ", id_data.shape, "\n")
print("identity_data[high_card_id_cols] after label_encoding: ",id_data[high_card_id_cols], "\n")

identity_data.shape before label-encoding:  (286140, 30) 

identity_data.shape after label-encoding:  (286140, 30) 

identity_data[high_card_id_cols] after label_encoding:                 id_30  id_31  id_33  DeviceInfo
TransactionID                                 
2987004            7    161    268        1565
2987008           70    130     80        2693
2987010           49     46    216        2526
2987011           49     46    216        2526
2987016           26     46     68        1170
...              ...    ...    ...         ...
4170230           49     74    216        2165
4170233           49     74    216        2106
4170234           63    129     80        2693
4170236           49     22    216         141
4170239           49    166    216        1529

[286140 rows x 4 columns] 



In [176]:
print(trn_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Columns: 223 entries, TransactionDT to V321
dtypes: float16(188), float32(23), object(12)
memory usage: 598.5+ MB
None


In [177]:
print(id_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 30 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_01       286140 non-null  float16
 1   id_02       286140 non-null  float32
 2   id_03       286140 non-null  float16
 3   id_04       286140 non-null  float16
 4   id_05       286140 non-null  float16
 5   id_06       286140 non-null  float16
 6   id_09       286140 non-null  float16
 7   id_10       286140 non-null  float16
 8   id_11       286140 non-null  float16
 9   id_12       286140 non-null  object 
 10  id_13       286140 non-null  float16
 11  id_14       286140 non-null  float16
 12  id_15       286140 non-null  object 
 13  id_16       286140 non-null  object 
 14  id_17       286140 non-null  float16
 15  id_19       286140 non-null  float16
 16  id_20       286140 non-null  float16
 17  id_28       286140 non-null  object 
 18  id_29       286140 non-null  object 


In [178]:
# Onehot-Encode for features with low cardinality

print("shape before encoding: ", trn_data.shape, "\n")
print("columns to encode: ", low_card_trans_cols, "\n")
print("transaction_data.columns.to_list() before encoding: ", trn_data.columns.to_list(), "\n")


# the onehot encoding
low_card_trans_encoded = pd.get_dummies(trn_data[low_card_trans_cols], dummy_na = False)
trn_data.drop(columns = low_card_trans_cols, inplace = True)

print("shape after encoding: ", trn_data.shape, "\n\n")
print("shape of new dataframe: ", low_card_trans_encoded.shape, "\n\n")
print("newly generated columns: ", low_card_trans_encoded.columns, "\n")
print("low_card_trans_encoded.info(): ", low_card_trans_encoded.info(),"\n")
print("transaction_data.columns.to_list() after encoding: ", trn_data.columns.to_list(), "\n")

shape before encoding:  (1097231, 223) 

columns to encode:  ['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M6', 'M7', 'M8', 'M9'] 

transaction_data.columns.to_list() before encoding:  ['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'P_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V

In [179]:
print("shape before encoding: ", id_data.shape, "\n")
print("columns to encode: ", low_card_id_cols, "\n")

# this line does the onehot encoding
low_card_id_encoded = pd.get_dummies(id_data[low_card_id_cols], dummy_na = False)
id_data.drop(columns = low_card_id_cols, inplace = True)


print("shape after encoding: ", id_data.shape, "\n\n")
print("shape of new dataframe: ", low_card_id_encoded.shape, "\n\n")
print("newly generated columns: ", low_card_id_encoded.columns, "\n")
print("low_card_id_encoded.info(): ", low_card_id_encoded.info())

shape before encoding:  (286140, 30) 

columns to encode:  ['id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType'] 

shape after encoding:  (286140, 19) 


shape of new dataframe:  (286140, 25) 


newly generated columns:  Index(['id_12_Found', 'id_12_NotFound', 'id_15_Found', 'id_15_New',
       'id_15_Unknown', 'id_16_Found', 'id_16_NotFound', 'id_28_Found',
       'id_28_New', 'id_29_Found', 'id_29_NotFound', 'id_34_match_status:-1',
       'id_34_match_status:0', 'id_34_match_status:1', 'id_34_match_status:2',
       'id_35_F', 'id_35_T', 'id_36_F', 'id_36_T', 'id_37_F', 'id_37_T',
       'id_38_F', 'id_38_T', 'DeviceType_desktop', 'DeviceType_mobile'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   id_12_Found            286140 non-nu

In [180]:
print(trn_data.isnull().sum().sum(), "\n")
print(low_card_trans_encoded.isnull().sum().sum())

0 

0


In [181]:
print(id_data.isnull().sum().sum(), "\n")
print(low_card_id_encoded.isnull().sum().sum())

0 

0


In [182]:
print(trn_data.info(), "\n")
print(low_card_trans_encoded.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Columns: 212 entries, TransactionDT to V321
dtypes: float16(188), float32(23), object(1)
memory usage: 506.5+ MB
None 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Data columns (total 30 columns):
 #   Column                  Non-Null Count    Dtype
---  ------                  --------------    -----
 0   ProductCD_C             1097231 non-null  uint8
 1   ProductCD_H             1097231 non-null  uint8
 2   ProductCD_R             1097231 non-null  uint8
 3   ProductCD_S             1097231 non-null  uint8
 4   ProductCD_W             1097231 non-null  uint8
 5   card4_american express  1097231 non-null  uint8
 6   card4_discover          1097231 non-null  uint8
 7   card4_mastercard        1097231 non-null  uint8
 8   card4_visa              1097231 non-null  uint8
 9   card6_charge card       1097231 non-null  uint8
 10  card6_credit            1097231 no

In [183]:
print(id_data.info(), "\n")
print(low_card_id_encoded.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_01       286140 non-null  float16
 1   id_02       286140 non-null  float32
 2   id_03       286140 non-null  float16
 3   id_04       286140 non-null  float16
 4   id_05       286140 non-null  float16
 5   id_06       286140 non-null  float16
 6   id_09       286140 non-null  float16
 7   id_10       286140 non-null  float16
 8   id_11       286140 non-null  float16
 9   id_13       286140 non-null  float16
 10  id_14       286140 non-null  float16
 11  id_17       286140 non-null  float16
 12  id_19       286140 non-null  float16
 13  id_20       286140 non-null  float16
 14  id_30       286140 non-null  int64  
 15  id_31       286140 non-null  int64  
 16  id_32       286140 non-null  float16
 17  id_33       286140 non-null  int64  
 18  DeviceInfo  286140 non-null  int64  


In [184]:
# Concat encoded dataframes

print("transaction_data.shape before concatting: ", trn_data.shape, "\n")
print("low_card_trans_encoded.shape before concatting: ", low_card_trans_encoded.shape, "\n")

trn_concatted = pd.concat([trn_data, low_card_trans_encoded], axis = 1)

print("transaction_concatted.shape after concatting: ", trn_concatted.shape, "\n")
print("transaction_concatted.columns after concatting: ", trn_concatted.columns, "\n")

#del low_card_trans_encoded
#del transaction_data

print(trn_concatted.info())

transaction_data.shape before concatting:  (1097231, 212) 

low_card_trans_encoded.shape before concatting:  (1097231, 30) 

transaction_concatted.shape after concatting:  (1097231, 242) 

transaction_concatted.columns after concatting:  Index(['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5',
       'addr1', 'addr2', 'dist1', 'P_emaildomain',
       ...
       'M4_M1', 'M4_M2', 'M6_F', 'M6_T', 'M7_F', 'M7_T', 'M8_F', 'M8_T',
       'M9_F', 'M9_T'],
      dtype='object', length=242) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Columns: 242 entries, TransactionDT to M9_T
dtypes: float16(188), float32(23), object(1), uint8(30)
memory usage: 537.9+ MB
None


In [185]:
print("identity_data.shape before concatting: ", id_data.shape, "\n")
print("low_card_id_encoded.shape before concatting: ", low_card_id_encoded.shape, "\n")

id_concatted = pd.concat([id_data, low_card_id_encoded], axis = 1)

print("identity_concatted.shape after concatting: ", id_concatted.shape, "\n")
print("identity_concatted.columns after concatting: ", id_concatted.columns, "\n")

#del low_card_id_encoded
#del identity_data

print(id_concatted.info())

identity_data.shape before concatting:  (286140, 19) 

low_card_id_encoded.shape before concatting:  (286140, 25) 

identity_concatted.shape after concatting:  (286140, 44) 

identity_concatted.columns after concatting:  Index(['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
       'id_11', 'id_13', 'id_14', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31',
       'id_32', 'id_33', 'DeviceInfo', 'id_12_Found', 'id_12_NotFound',
       'id_15_Found', 'id_15_New', 'id_15_Unknown', 'id_16_Found',
       'id_16_NotFound', 'id_28_Found', 'id_28_New', 'id_29_Found',
       'id_29_NotFound', 'id_34_match_status:-1', 'id_34_match_status:0',
       'id_34_match_status:1', 'id_34_match_status:2', 'id_35_F', 'id_35_T',
       'id_36_F', 'id_36_T', 'id_37_F', 'id_37_T', 'id_38_F', 'id_38_T',
       'DeviceType_desktop', 'DeviceType_mobile'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 44 columns

In [186]:
print("transaction_concatted.shape before splitting up: ", trn_concatted.shape, "\n")

# shape of train_transaction was (590540, 393), 
# shape of test_transaction  was (506691, 392)
train_trn = trn_concatted.iloc[0:590540]
test_trn = trn_concatted.iloc[590540:]

print("train_transaction.shape after splitting up: ", train_trn.shape, "\n")
print("test_transaction.shape after splitting up: ", test_trn.shape, "\n")

transaction_concatted.shape before splitting up:  (1097231, 242) 

train_transaction.shape after splitting up:  (590540, 242) 

test_transaction.shape after splitting up:  (506691, 242) 



In [187]:
print("identity_concatted.shape before splitting up: ", id_concatted.shape, "\n")

# shape of train_identity was  (144233, 40)
# shape of test_identity  was  (141907, 40)
train_id = id_concatted.iloc[0:144233]
test_id = id_concatted.iloc[144233:]

print("train_identity.shape after splitting up: ", train_id.shape, "\n")
print("test_identity.shape after splitting up: ", test_id.shape, "\n")

identity_concatted.shape before splitting up:  (286140, 44) 

train_identity.shape after splitting up:  (144233, 44) 

test_identity.shape after splitting up:  (141907, 44) 



In [188]:
print("train_transaction.shape before concatting: ", train_trn.shape, "\n")
print("train_identity.shape before concatting: ", train_id.shape, "\n")

train_data  = pd.concat([train_trn, train_id], axis = 1)

print("train_data.shape: ", train_data.shape)

train_transaction.shape before concatting:  (590540, 242) 

train_identity.shape before concatting:  (144233, 44) 

train_data.shape:  (590540, 286)


In [189]:
train_data = train_data.drop(columns =['P_emaildomain'])

In [190]:
counter = 0

for i in train_data.columns:
    
    summ = train_data[i].isnull().sum()
    print(i, summ)
    if summ > 0:
        counter += 1
        
print("\n number of columns with missing values: ", counter)

TransactionDT 0
TransactionAmt 0
card1 0
card2 0
card3 0
card5 0
addr1 0
addr2 0
dist1 0
C1 0
C2 0
C3 0
C4 0
C5 0
C6 0
C7 0
C8 0
C9 0
C10 0
C11 0
C12 0
C13 0
C14 0
D1 0
D2 0
D3 0
D4 0
D5 0
D10 0
D11 0
D15 0
V1 0
V2 0
V3 0
V4 0
V5 0
V6 0
V7 0
V8 0
V9 0
V10 0
V11 0
V12 0
V13 0
V14 0
V15 0
V16 0
V17 0
V18 0
V19 0
V20 0
V21 0
V22 0
V23 0
V24 0
V25 0
V26 0
V27 0
V28 0
V29 0
V30 0
V31 0
V32 0
V33 0
V34 0
V35 0
V36 0
V37 0
V38 0
V39 0
V40 0
V41 0
V42 0
V43 0
V44 0
V45 0
V46 0
V47 0
V48 0
V49 0
V50 0
V51 0
V52 0
V53 0
V54 0
V55 0
V56 0
V57 0
V58 0
V59 0
V60 0
V61 0
V62 0
V63 0
V64 0
V65 0
V66 0
V67 0
V68 0
V69 0
V70 0
V71 0
V72 0
V73 0
V74 0
V75 0
V76 0
V77 0
V78 0
V79 0
V80 0
V81 0
V82 0
V83 0
V84 0
V85 0
V86 0
V87 0
V88 0
V89 0
V90 0
V91 0
V92 0
V93 0
V94 0
V95 0
V96 0
V97 0
V98 0
V99 0
V100 0
V101 0
V102 0
V103 0
V104 0
V105 0
V106 0
V107 0
V108 0
V109 0
V110 0
V111 0
V112 0
V113 0
V114 0
V115 0
V116 0
V117 0
V118 0
V119 0
V120 0
V121 0
V122 0
V123 0
V124 0
V125 0
V126 0
V127 0
V128 0
V129 

In [191]:
print("test_transaction.shape before concatting: ", test_trn.shape, "\n")
print("test_identity.shape before concatting: ", test_id.shape, "\n")

test_data  = pd.concat([test_trn, test_id], axis = 1)

print("test_data.shape: ", test_data.shape)

test_transaction.shape before concatting:  (506691, 242) 

test_identity.shape before concatting:  (141907, 44) 

test_data.shape:  (506691, 286)


In [192]:
test_data = test_data.drop(columns =['P_emaildomain'])

In [193]:
counter = 0

for i in test_data.columns:
    
    summ = test_data[i].isnull().sum()
    print(i, summ)
    if summ > 0:
        counter += 1
        
print("\n number of columns with missing values: ", counter)

TransactionDT 0
TransactionAmt 0
card1 0
card2 0
card3 0
card5 0
addr1 0
addr2 0
dist1 0
C1 0
C2 0
C3 0
C4 0
C5 0
C6 0
C7 0
C8 0
C9 0
C10 0
C11 0
C12 0
C13 0
C14 0
D1 0
D2 0
D3 0
D4 0
D5 0
D10 0
D11 0
D15 0
V1 0
V2 0
V3 0
V4 0
V5 0
V6 0
V7 0
V8 0
V9 0
V10 0
V11 0
V12 0
V13 0
V14 0
V15 0
V16 0
V17 0
V18 0
V19 0
V20 0
V21 0
V22 0
V23 0
V24 0
V25 0
V26 0
V27 0
V28 0
V29 0
V30 0
V31 0
V32 0
V33 0
V34 0
V35 0
V36 0
V37 0
V38 0
V39 0
V40 0
V41 0
V42 0
V43 0
V44 0
V45 0
V46 0
V47 0
V48 0
V49 0
V50 0
V51 0
V52 0
V53 0
V54 0
V55 0
V56 0
V57 0
V58 0
V59 0
V60 0
V61 0
V62 0
V63 0
V64 0
V65 0
V66 0
V67 0
V68 0
V69 0
V70 0
V71 0
V72 0
V73 0
V74 0
V75 0
V76 0
V77 0
V78 0
V79 0
V80 0
V81 0
V82 0
V83 0
V84 0
V85 0
V86 0
V87 0
V88 0
V89 0
V90 0
V91 0
V92 0
V93 0
V94 0
V95 0
V96 0
V97 0
V98 0
V99 0
V100 0
V101 0
V102 0
V103 0
V104 0
V105 0
V106 0
V107 0
V108 0
V109 0
V110 0
V111 0
V112 0
V113 0
V114 0
V115 0
V116 0
V117 0
V118 0
V119 0
V120 0
V121 0
V122 0
V123 0
V124 0
V125 0
V126 0
V127 0
V128 0
V129 

In [194]:
# XGBClassifier

from xgboost import XGBClassifier
clf = XGBClassifier(objective = 'binary:logistic', gamma = 0.05, colsample_bytree = 0.5, 
                    eval_metric = 'auc', n_estimators = 1350, max_depth = 8,
                    min_child_weight = 2, learning_rate = 0.02,subsample = 0.8,
                    n_jobs = -1, silent = False, verbosity = 0)

In [195]:
#!pip install keras

In [196]:
#import tenserflow as tf
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
#with tpu_strategy.scope():
 #   clf = XGBClassifier(objective = 'binary:logistic', gamma = 0.05, colsample_bytree = 0.5, 
  #                  eval_metric = 'auc', n_estimators = 1350, max_depth = 8,
   #                 min_child_weight = 2, learning_rate = 0.02,subsample = 0.8,
    #                n_jobs = -1, silent = False, verbosity = 0)

# train model normally
#model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

In [197]:
clf.fit(train_data, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0.05, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.02, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
              missing=nan, monotone_constraints='()', n_estimators=1350,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [200]:
sample_submission = pd.read_csv('/kaggle/input/ieee-fraud-detection/sample_submission.csv', index_col='TransactionID')

sample_submission['isFraud'] = clf.predict_proba(test_data)[:,1]
sample_submission.to_csv('simple_xgboost.csv')

In [199]:
# make predictions for test data
#y_pred = model.predict(test_data)
#predictions = [round(value) for value in y_pred]
# evaluate predictions
#accuracy = accuracy_score(y_test, predictions)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))